In [1]:
import pandas as pd
import os
import plotly.express as px
import numpy as np

In [2]:
# select the correct python binary
python = 'python3'
if os.system('python3 --version') != 0:
    python = 'python'

n = 50
# run scraper for n number of pages
!{python} scraper.py {n}

Python 3.10.8


In [3]:
df = pd.read_csv('all_cars.csv', delimiter=';', index_col=0)
df.head(10)

,Title:,Description:,Price:,Odometer:,Fuel Type:,Colour:,Engine Size:,Transmission:,Body Type:,Owners:,Doors:,Tax Expiry:,NCT Expiry:,Dealer:,Address:,Phone:,Franchise:,Website:
0,VOLKSWAGEN PASSAT,2012 2.0 TDI CC BLUEMOTION 138BHP 5 SEATS 4DR,"€8,295","133,600 kms",Diesel,Beige,2.0 L,Manual,Saloon,1,4,Apr-2023,Aug-2023,0838704977,NaN,NaN,NaN,NaN
1,OPEL INSIGNIA,2010 S 2.0 CDTI 130PS ECO 5DR,"€3,800","219,000 kms",Diesel,Beige,2.0 L,Manual,Saloon,5,5,Jan-2023,Jan-2024,0831274143,NaN,NaN,NaN,NaN
2,HYUNDAI TUCSON,2016 EXECUTIVE (Hi-Spec Model),"€17,950",0 kms,Diesel,Black,1.7 L,Manual,MPV,1,5,Apr-2022,Sep-2024,Barry Martin Motors Ltd,Irishtown Filling Station Ashe Road ...,044-9339962,NaN,NaN
3,SUZUKI CELERIO,2017 CELERIO 1.0 Petrol 5DR,"€9,950","119,003 kms",Petrol,Grey,1.0 L,Manual,Hatchback,1,5,Dec-2022,Jul-2023,Barry Martin Motors Ltd,Irishtown Filling Station Ashe Road ...,044-9339962,NaN,NaN
4,VOLKSWAGEN TIGUAN,2017 Comfort Line 2.0 TDI 150 HP,"€22,950",0 kms,Diesel,Grey,2.0 L,Manual,MPV,3,5,Dec-2022,Jan-2023,Barry Martin Motors Ltd,Irishtown Filling Station Ashe Road ...,044-9339962,NaN,NaN
5,HONDA VEZEL,2015 VEZEL 1.5 DAA-RU3 5 DR AUTO,€0,"101,499 kms",Petrol Hybrid,White,1.5 L,Automatic,MPV,1,-,-,-,Kearys Carstore Limerick,Unit 6B Eastpoint Retail Park Ballys...,061-326500,NaN,NaN
6,MERCEDES-BENZ CLA CLASS,2017 CLA 180 PET URBAN,"€24,990","89,520 kms",Petrol,Black,1.6 L,Manual,Saloon,1,4,Dec-2022,Jan-2023,Kearys Carstore Limerick,Unit 6B Eastpoint Retail Park Ballys...,061-326500,NaN,NaN
7,AUDI TT,2018 2.0TDI 184HP S-Line,"€36,950","77,998 kms",Diesel,Black,2.0 L,Manual,Coupe,1,-,-,Jun-2022,Lyons of Limerick,Ballysimon Road Crabb's Land,061-439800,NaN,NaN
8,BMW 5 SERIES,2017 520d SE Auto,"€29,995","148,341 kms",Diesel,Blue,2.0 L,Automatic,Saloon,2,4,Oct-2023,Feb-2023,Colm Quinn BMW Drogheda,M1 Retail Park Junction 10 Drogheda,041-6874888,NaN,NaN
9,BMW 2 SERIES,2022 218d Gran Coupe Sport Auto,"€43,995","10,718 kms",Diesel,Grey,2.0 L,Automatic,Saloon,1,4,Dec-2022,Jan-2026,Colm Quinn BMW Drogheda,M1 Retail Park Junction 10 Drogheda,041-6874888,NaN,NaN


In [4]:
# Sanitise the data
df['Year:'] = df['Description:'].str.split(' ', expand=True)[0].apply(pd.to_numeric)
df['Title:'] = df['Title:'].str.replace('LAND ROVER', 'LAND-ROVER')
df['Manufacturer:'] = df['Title:'].str.split(' ', expand=True)[0]
df['Model:'] = df['Title:'].str.split(' ', expand=False).apply(lambda x: ''.join(x[1:]))
df['Price:'] = df['Price:'].str.replace('€', '')
df['Price:'] = df['Price:'].str.replace(',', '')
df['Price:'] = df['Price:'].apply(pd.to_numeric)

In [5]:
df_year_price = df[['Year:', 'Price:']].copy()
df_year_price

,Year:,Price:
0,2012,8295
1,2010,3800
2,2016,17950
3,2017,9950
4,2017,22950
...,...,...
894,2014,0
895,2013,0
896,2011,0
897,2013,10995


In [6]:
df_year_price['Price:'] = df_year_price['Price:'][df_year_price['Price:'] > 1]
df_year_price

,Year:,Price:
0,2012,8295.0
1,2010,3800.0
2,2016,17950.0
3,2017,9950.0
4,2017,22950.0
...,...,...
894,2014,NaN
895,2013,NaN
896,2011,NaN
897,2013,10995.0


In [7]:
px.scatter(x=df_year_price['Year:'], y=df_year_price['Price:'], labels={'x': 'Year', 'y': 'Price'})

In [8]:
px.box(x=df_year_price['Year:'], y=df_year_price['Price:'], labels={'x': 'Year', 'y': 'Price'})

In [9]:
# Get rid of cars with prices tagged as 0 or 1
df = df[df['Price:'] > 1]

In [10]:
df_groupby_year = df.groupby(df['Year:'])
df_groupby_year.describe()

Price:                                                          \
       count          mean           std      min       25%      50%   
Year:                                                                  
2005     2.0   2600.000000    494.974747   2250.0   2425.00   2600.0   
2007    10.0   4212.400000   3458.781712   1950.0   2622.50   2997.0   
2008     7.0   4807.000000   3122.866151   2250.0   2850.00   3800.0   
2009     7.0   6384.000000   2477.426756   3495.0   4972.50   5999.0   
2010    10.0   5584.400000   1712.960154   3800.0   4337.25   5225.0   
2011    14.0   8017.071429   2817.179044   2750.0   5950.00   8350.0   
2012    11.0   9117.272727   3750.143573   5750.0   6900.00   7995.0   
2013    31.0  11112.064516   3926.019672   5750.0   8100.00   9950.0   
2014    36.0  13676.638889   4948.269407   7500.0   9995.00  12850.0   
2015    49.0  15000.632653   6548.215710   7995.0  10999.00  13950.0   
2016    66.0  19587.227273   7842.517841   7950.0  14562.50  17920.0   
2017    86.0  20269.744186   8679.592619   9495.0  14908.75  17950.0   
2018    51.0  24824.568627  11902.241430  11495.0  16995.00  21440.0   
2019    79.0  26737.329114  13953.737415  11995.0  18445.00  24700.0   
2020    58.0  36618.775862  18844.270945  17595.0  24087.50  32472.5   
2021    49.0  37194.183673  21506.749875  15895.0  22595.00  29995.0   
2022    66.0  39143.712121  18295.744447  15650.0  27450.00  30585.0   
2023    38.0  38670.394737  11973.753137  23450.0  28425.50  36417.5   

                           
            75%       max  
Year:                      
2005    2775.00    2950.0  
2007    3808.75   13500.0  
2008    5474.50   10950.0  
2009    7124.50   11000.0  
2010    5983.75    8950.0  
2011   10750.00   10950.0  
2012    9825.00   18500.0  
2013   13200.00   20900.0  
2014   14912.50   32950.0  
2015   16450.00   45528.0  
2016   23371.00   46950.0  
2017   22950.00   65900.0  
2018   26447.50   79950.0  
2019   29100.00   80900.0  
2020   43650.00  135000.0  
2021   47995.00  107950.0  
2022   47745.00   96995.0  
2023   46559.75   69950.0

In [11]:
avg_price_per_year = df_groupby_year['Price:'].agg(['mean'])
px.bar(avg_price_per_year, labels={'value': 'Average Price', 'Year:': 'Year'})

In [12]:
df2 = df[['Year:', 'Price:', 'Engine Size:', 'Transmission:', 'Manufacturer:', 'Doors:', 'Odometer:']].copy()
df2 = df2[df2['Doors:'] != '-']
df2 = df2[df2['Transmission:'] != 'Other']
df2

,Year:,Price:,Engine Size:,Transmission:,Manufacturer:,Doors:,Odometer:
0,2012,8295,2.0 L,Manual,VOLKSWAGEN,4,"133,600 kms"
1,2010,3800,2.0 L,Manual,OPEL,5,"219,000 kms"
2,2016,17950,1.7 L,Manual,HYUNDAI,5,0 kms
3,2017,9950,1.0 L,Manual,SUZUKI,5,"119,003 kms"
4,2017,22950,2.0 L,Manual,VOLKSWAGEN,5,0 kms
...,...,...,...,...,...,...,...
861,2020,61175,0.0 L,Automatic,AUDI,5,"37,858 kms"
862,2020,22950,1.0 L,Manual,HYUNDAI,5,"13,657 kms"
863,2021,23500,1.0 L,Manual,SKODA,4,"27,529 kms"
897,2013,10995,1.6 L,Manual,NISSAN,5,"110,003 kms"


In [31]:
top_manufacturers = df2['Manufacturer:'].value_counts()[:6].index.tolist()
top_manufacturers

['VOLKSWAGEN', 'RENAULT', 'FORD', 'NISSAN', 'BMW', 'TOYOTA']

In [14]:
df3 = df2[df2['Manufacturer:'].isin(top_manufacturers)]
px.scatter(df3, x='Year:', y='Price:', color='Doors:', facet_row='Transmission:', facet_col='Manufacturer:')

In [15]:
df4 = df3.pivot_table(values=['Price:'], index=['Manufacturer:', 'Transmission:', 'Doors:', 'Year:'], aggfunc=np.mean)
df4 = df4.reset_index(level=[0, 1, 2, 3])

In [16]:
px.line(df4, x='Year:', y='Price:', color='Transmission:', facet_row='Doors:', facet_col='Manufacturer:')

In [17]:
df5 = df3.pivot_table(values=['Price:'], index=['Year:', 'Transmission:', 'Manufacturer:'], aggfunc='count')
df5 = df5.reset_index(level=[0, 1, 2])
df5 = df5.rename(columns={'Price:': 'Count:'})
df5 = df5.sort_values(by='Year:')
df5

,Year:,Transmission:,Manufacturer:,Count:
0,2005,Automatic,FORD,1
1,2005,Manual,VOLKSWAGEN,1
2,2007,Automatic,FORD,1
3,2007,Manual,NISSAN,1
4,2007,Manual,VOLKSWAGEN,1
...,...,...,...,...
90,2022,Manual,NISSAN,1
91,2022,Manual,RENAULT,22
92,2022,Manual,VOLKSWAGEN,3
93,2023,Automatic,VOLKSWAGEN,1


In [18]:
px.pie(df5, names='Year:', values='Count:', facet_row='Transmission:', facet_col='Manufacturer:')

In [19]:
df_export = df[['Manufacturer:', 'Model:', 'Description:', 'Price:', 'Odometer:', 'Fuel Type:', 'Colour:', 'Engine Size:', 'Transmission:', 'Body Type:', 'Doors:']]
df_export.to_csv('cars-ie-export.csv', sep=';')
df_export

,Manufacturer:,Model:,Description:,Price:,Odometer:,Fuel Type:,Colour:,Engine Size:,Transmission:,Body Type:,Doors:
0,VOLKSWAGEN,PASSAT,2012 2.0 TDI CC BLUEMOTION 138BHP 5 SEATS 4DR,8295,"133,600 kms",Diesel,Beige,2.0 L,Manual,Saloon,4
1,OPEL,INSIGNIA,2010 S 2.0 CDTI 130PS ECO 5DR,3800,"219,000 kms",Diesel,Beige,2.0 L,Manual,Saloon,5
2,HYUNDAI,TUCSON,2016 EXECUTIVE (Hi-Spec Model),17950,0 kms,Diesel,Black,1.7 L,Manual,MPV,5
3,SUZUKI,CELERIO,2017 CELERIO 1.0 Petrol 5DR,9950,"119,003 kms",Petrol,Grey,1.0 L,Manual,Hatchback,5
4,VOLKSWAGEN,TIGUAN,2017 Comfort Line 2.0 TDI 150 HP,22950,0 kms,Diesel,Grey,2.0 L,Manual,MPV,5
...,...,...,...,...,...,...,...,...,...,...,...
861,AUDI,E-TRON,2020 E-Tron Technik 50 Quattro Technik 71.2 kW...,61175,"37,858 kms",Electric,Silver,0.0 L,Automatic,SUV,5
862,HYUNDAI,KONA,2020 * JUST ARRIVED * TINY MILEAGE KONA 1.0 T-...,22950,"13,657 kms",Petrol,Red,1.0 L,Manual,MPV,5
863,SKODA,KAMIQ,2021 1.0TSI 115hp Style,23500,"27,529 kms",Petrol,Black,1.0 L,Manual,Hatchback,4
897,NISSAN,JUKE,2013 1.6 XE,10995,"110,003 kms",Petrol,White,1.6 L,Manual,Hatchback,5


In [28]:
df['Engine Size:'] = df['Engine Size:'].apply(lambda x: float(x.split(' ')[0]))
df['Odometer:'] = df['Odometer:'].apply(lambda x: int(x.split(' ')[0].replace(',', '').replace('.', '')))

In [29]:
def filter(df):
    df = df[df['Odometer:'] > 0]
    df = df[df['Engine Size:'] > 0]
    return df

In [30]:
filter(df.copy())

,Title:,Description:,Price:,Odometer:,Fuel Type:,Colour:,Engine Size:,Transmission:,Body Type:,Owners:,...,Tax Expiry:,NCT Expiry:,Dealer:,Address:,Phone:,Franchise:,Website:,Year:,Manufacturer:,Model:
0,VOLKSWAGEN PASSAT,2012 2.0 TDI CC BLUEMOTION 138BHP 5 SEATS 4DR,8295,133600,Diesel,Beige,2.0,Manual,Saloon,1,...,Apr-2023,Aug-2023,0838704977,NaN,NaN,NaN,NaN,2012,VOLKSWAGEN,PASSAT
1,OPEL INSIGNIA,2010 S 2.0 CDTI 130PS ECO 5DR,3800,219000,Diesel,Beige,2.0,Manual,Saloon,5,...,Jan-2023,Jan-2024,0831274143,NaN,NaN,NaN,NaN,2010,OPEL,INSIGNIA
3,SUZUKI CELERIO,2017 CELERIO 1.0 Petrol 5DR,9950,119003,Petrol,Grey,1.0,Manual,Hatchback,1,...,Dec-2022,Jul-2023,Barry Martin Motors Ltd,Irishtown Filling Station Ashe Road ...,044-9339962,NaN,NaN,2017,SUZUKI,CELERIO
6,MERCEDES-BENZ CLA CLASS,2017 CLA 180 PET URBAN,24990,89520,Petrol,Black,1.6,Manual,Saloon,1,...,Dec-2022,Jan-2023,Kearys Carstore Limerick,Unit 6B Eastpoint Retail Park Ballys...,061-326500,NaN,NaN,2017,MERCEDES-BENZ,CLACLASS
7,AUDI TT,2018 2.0TDI 184HP S-Line,36950,77998,Diesel,Black,2.0,Manual,Coupe,1,...,-,Jun-2022,Lyons of Limerick,Ballysimon Road Crabb's Land,061-439800,NaN,NaN,2018,AUDI,TT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,AUDI A5,2020 40TDI 190HP S tronic SE €587 p/m PCP,46475,55912,Diesel,Black,2.0,Automatic,Hatchback,1,...,Jan-2023,Jan-2024,Audi Wexford,Crosstown Wexford,053-9100010,Audi,www.audiwexford.ie,2020,AUDI,A5
862,HYUNDAI KONA,2020 * JUST ARRIVED * TINY MILEAGE KONA 1.0 T-...,22950,13657,Petrol,Red,1.0,Manual,MPV,1,...,Dec-2022,Jan-2024,McCoy Motors Ltd,Exit 4A N4 Lucan,01-6241223,SEAT,www.mccoymotors.ie,2020,HYUNDAI,KONA
863,SKODA KAMIQ,2021 1.0TSI 115hp Style,23500,27529,Petrol,Black,1.0,Manual,Hatchback,1,...,Dec-2022,Jan-2025,Ballybrittas Motors,Ballybrittas Portlaoise,057-8626479,Skoda,www.ballybrittasmotors.ie,2021,SKODA,KAMIQ
897,NISSAN JUKE,2013 1.6 XE,10995,110003,Petrol,White,1.6,Manual,Hatchback,1,...,Jan-2023,Mar-2023,Windsor Clonee Nissan & Peugeot,Main Street Clonee,01-5077354,NaN,NaN,2013,NISSAN,JUKE
